In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from functools import reduce
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

In [2]:
caracteristics = pd.read_csv('C:/Users/mnhelal/OneDrive - Université Paris 1 Panthéon-Sorbonne/Bureau/Paris/Algorithmique et Python/archive/caracteristics.csv',encoding='latin-1', sep=",") 
places = pd.read_csv('C:/Users/mnhelal/OneDrive - Université Paris 1 Panthéon-Sorbonne/Bureau/Paris/Algorithmique et Python/archive/places.csv', sep=",")
users = pd.read_csv('C:/Users/mnhelal/OneDrive - Université Paris 1 Panthéon-Sorbonne/Bureau/Paris/Algorithmique et Python/archive/users.csv', sep=",")
vehicules = pd.read_csv('C:/Users/mnhelal/OneDrive - Université Paris 1 Panthéon-Sorbonne/Bureau/Paris/Algorithmique et Python/archive/vehicles.csv', sep=",")

C:\Users\mnhelal\AppData\Local\Temp\ipykernel_8584\3722665817.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  caracteristics = pd.read_csv('C:/Users/mnhelal/OneDrive - Université Paris 1 Panthéon-Sorbonne/Bureau/Paris/Algorithmique et Python/archive/caracteristics.csv',encoding='latin-1', sep=",")
C:\Users\mnhelal\AppData\Local\Temp\ipykernel_8584\3722665817.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  places = pd.read_csv('C:/Users/mnhelal/OneDrive - Université Paris 1 Panthéon-Sorbonne/Bureau/Paris/Algorithmique et Python/archive/places.csv', sep=",")


In [3]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [4]:
places = places[places.nbv <= 10]
places['prof'] = places['prof'].replace(0.0, 1.0)

places['plan'] = places['plan'].replace(0.0, 1.0)


places['surf'] = places['surf'].replace(0.0, 1.0)



places['circ'] = places['circ'].replace(0.0, 2.0)


In [5]:
domtom = [971,972,973,974,976,975] #But this doesn't include les Antilles (Martinique or Guadeloupe) cause they're not departements

caracteristics = caracteristics[~caracteristics.dep.isin(domtom)]


In [6]:
caracteristics['lum'] = caracteristics['lum'].replace(0, 1)

caracteristics['int'] = caracteristics['int'].replace(0, 1)

In [7]:
# Dropping les Antilles so we can stay on only le Métropole
caracteristics = caracteristics[caracteristics.gps != "A"]
caracteristics

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep
0,201600000001,16,2,1,1445,1,2,1,8.0,3.0,5.0,"46, rue Sonneville",M,0.0,0.0,590
1,201600000002,16,3,16,1800,1,2,6,1.0,6.0,5.0,1a rue du cimetière,M,0.0,0.0,590
2,201600000003,16,7,13,1900,1,1,1,1.0,6.0,11.0,NaN,M,0.0,0.0,590
3,201600000004,16,8,15,1930,2,2,1,7.0,3.0,477.0,52 rue victor hugo,M,0.0,0.0,590
4,201600000005,16,12,23,1100,1,2,3,1.0,3.0,11.0,rue Joliot curie,M,0.0,0.0,590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837479,200500084521,5,12,30,2000,5,1,1,1.0,2.0,55.0,A7-Sens Marseille vers L,NaN,NaN,NaN,130
837480,200500084522,5,12,31,1230,1,1,1,1.0,6.0,71.0,A51-Sens Aix en Provence,NaN,NaN,NaN,130
837481,200500084523,5,12,31,1412,1,1,1,8.0,7.0,55.0,A7-Sens Marseille vers L,NaN,NaN,NaN,130
837482,200500084524,5,12,31,2020,3,1,1,2.0,7.0,55.0,A55-Sens Martigues vers,NaN,NaN,NaN,130


In [8]:
#le merge 
from functools import reduce
Merge1 = [caracteristics, places]
Merge_1 = reduce(lambda left,right: pd.merge(left,right,on=['Num_Acc'],how='left'),Merge1)


Merge2 = [vehicules, users]
Merge_2 = reduce(lambda left,right: pd.merge(left,right,on=['Num_Acc','num_veh'],how='left'),Merge2)


Merge3 = [Merge_2, Merge_1]

accidents = reduce(lambda left,right: pd.merge(left,right,on=['Num_Acc'],how='left'),Merge3)

In [9]:
missing_values_table(accidents)

Your selected dataframe has 51 columns.
There are 47 columns that have missing values.


,Missing Values,% of Total Values
v2,1818456,95.8
v1,1167200,61.5
long,1090142,57.5
lat,1090132,57.5
gps,1084100,57.1
pr1,969029,51.1
pr,966532,50.9
adr,384301,20.3
voie,188989,10.0
place,121723,6.4


In [10]:
accidents = accidents.drop(['v1', 'v2', 'pr', 'pr1', 'lartpc', 'adr', 'voie','long','lat','gps','locp', 'actp', 'etatp','larrout','env1','vosp','place','manv','obsm','obs','senc'], axis=1)

In [11]:
for i in accidents.columns:
    accidents[i].fillna(accidents[i].mode()[0], inplace=True)
    print(accidents)

              Num_Acc  catv  occutc  choc num_veh  catu  grav  sexe  trajet  \
0        201600000001     7       0   1.0     B02   1.0   1.0   2.0     0.0   
1        201600000001     2       0   7.0     A01   1.0   3.0   1.0     9.0   
2        201600000002     7       0   1.0     A01   1.0   3.0   1.0     5.0   
3        201600000002     7       0   1.0     A01   2.0   3.0   1.0     0.0   
4        201600000002     7       0   1.0     A01   2.0   3.0   2.0     0.0   
...               ...   ...     ...   ...     ...   ...   ...   ...     ...   
1897335  200500087953     7       0   3.0     B01   1.0   1.0   1.0     0.0   
1897336  200500087953     2       0   1.0     A01   1.0   4.0   1.0     5.0   
1897337  200500087953     7       0   1.0     C01   1.0   1.0   1.0     0.0   
1897338  200500087954     7       0   0.0     A01   1.0   4.0   1.0     0.0   
1897339  200500087954     7       0   0.0     A01   2.0   4.0   2.0     0.0   

         secu  ...  com    dep  catr  circ  nbv  pr

              Num_Acc  catv  occutc  choc num_veh  catu  grav  sexe  trajet  \
0        201600000001     7       0   1.0     B02   1.0   1.0   2.0     0.0   
1        201600000001     2       0   7.0     A01   1.0   3.0   1.0     9.0   
2        201600000002     7       0   1.0     A01   1.0   3.0   1.0     5.0   
3        201600000002     7       0   1.0     A01   2.0   3.0   1.0     0.0   
4        201600000002     7       0   1.0     A01   2.0   3.0   2.0     0.0   
...               ...   ...     ...   ...     ...   ...   ...   ...     ...   
1897335  200500087953     7       0   3.0     B01   1.0   1.0   1.0     0.0   
1897336  200500087953     2       0   1.0     A01   1.0   4.0   1.0     5.0   
1897337  200500087953     7       0   1.0     C01   1.0   1.0   1.0     0.0   
1897338  200500087954     7       0   0.0     A01   1.0   4.0   1.0     0.0   
1897339  200500087954     7       0   0.0     A01   2.0   4.0   2.0     0.0   

         secu  ...  com    dep  catr  circ  nbv  pr

              Num_Acc  catv  occutc  choc num_veh  catu  grav  sexe  trajet  \
0        201600000001     7       0   1.0     B02   1.0   1.0   2.0     0.0   
1        201600000001     2       0   7.0     A01   1.0   3.0   1.0     9.0   
2        201600000002     7       0   1.0     A01   1.0   3.0   1.0     5.0   
3        201600000002     7       0   1.0     A01   2.0   3.0   1.0     0.0   
4        201600000002     7       0   1.0     A01   2.0   3.0   2.0     0.0   
...               ...   ...     ...   ...     ...   ...   ...   ...     ...   
1897335  200500087953     7       0   3.0     B01   1.0   1.0   1.0     0.0   
1897336  200500087953     2       0   1.0     A01   1.0   4.0   1.0     5.0   
1897337  200500087953     7       0   1.0     C01   1.0   1.0   1.0     0.0   
1897338  200500087954     7       0   0.0     A01   1.0   4.0   1.0     0.0   
1897339  200500087954     7       0   0.0     A01   2.0   4.0   2.0     0.0   

         secu  ...  com    dep  catr  circ  nbv  pr

              Num_Acc  catv  occutc  choc num_veh  catu  grav  sexe  trajet  \
0        201600000001     7       0   1.0     B02   1.0   1.0   2.0     0.0   
1        201600000001     2       0   7.0     A01   1.0   3.0   1.0     9.0   
2        201600000002     7       0   1.0     A01   1.0   3.0   1.0     5.0   
3        201600000002     7       0   1.0     A01   2.0   3.0   1.0     0.0   
4        201600000002     7       0   1.0     A01   2.0   3.0   2.0     0.0   
...               ...   ...     ...   ...     ...   ...   ...   ...     ...   
1897335  200500087953     7       0   3.0     B01   1.0   1.0   1.0     0.0   
1897336  200500087953     2       0   1.0     A01   1.0   4.0   1.0     5.0   
1897337  200500087953     7       0   1.0     C01   1.0   1.0   1.0     0.0   
1897338  200500087954     7       0   0.0     A01   1.0   4.0   1.0     0.0   
1897339  200500087954     7       0   0.0     A01   2.0   4.0   2.0     0.0   

         secu  ...  com    dep  catr  circ  nbv  pr

              Num_Acc  catv  occutc  choc num_veh  catu  grav  sexe  trajet  \
0        201600000001     7       0   1.0     B02   1.0   1.0   2.0     0.0   
1        201600000001     2       0   7.0     A01   1.0   3.0   1.0     9.0   
2        201600000002     7       0   1.0     A01   1.0   3.0   1.0     5.0   
3        201600000002     7       0   1.0     A01   2.0   3.0   1.0     0.0   
4        201600000002     7       0   1.0     A01   2.0   3.0   2.0     0.0   
...               ...   ...     ...   ...     ...   ...   ...   ...     ...   
1897335  200500087953     7       0   3.0     B01   1.0   1.0   1.0     0.0   
1897336  200500087953     2       0   1.0     A01   1.0   4.0   1.0     5.0   
1897337  200500087953     7       0   1.0     C01   1.0   1.0   1.0     0.0   
1897338  200500087954     7       0   0.0     A01   1.0   4.0   1.0     0.0   
1897339  200500087954     7       0   0.0     A01   2.0   4.0   2.0     0.0   

         secu  ...   com    dep  catr  circ  nbv  p

              Num_Acc  catv  occutc  choc num_veh  catu  grav  sexe  trajet  \
0        201600000001     7       0   1.0     B02   1.0   1.0   2.0     0.0   
1        201600000001     2       0   7.0     A01   1.0   3.0   1.0     9.0   
2        201600000002     7       0   1.0     A01   1.0   3.0   1.0     5.0   
3        201600000002     7       0   1.0     A01   2.0   3.0   1.0     0.0   
4        201600000002     7       0   1.0     A01   2.0   3.0   2.0     0.0   
...               ...   ...     ...   ...     ...   ...   ...   ...     ...   
1897335  200500087953     7       0   3.0     B01   1.0   1.0   1.0     0.0   
1897336  200500087953     2       0   1.0     A01   1.0   4.0   1.0     5.0   
1897337  200500087953     7       0   1.0     C01   1.0   1.0   1.0     0.0   
1897338  200500087954     7       0   0.0     A01   1.0   4.0   1.0     0.0   
1897339  200500087954     7       0   0.0     A01   2.0   4.0   2.0     0.0   

         secu  ...   com    dep  catr  circ  nbv  p

              Num_Acc  catv  occutc  choc num_veh  catu  grav  sexe  trajet  \
0        201600000001     7       0   1.0     B02   1.0   1.0   2.0     0.0   
1        201600000001     2       0   7.0     A01   1.0   3.0   1.0     9.0   
2        201600000002     7       0   1.0     A01   1.0   3.0   1.0     5.0   
3        201600000002     7       0   1.0     A01   2.0   3.0   1.0     0.0   
4        201600000002     7       0   1.0     A01   2.0   3.0   2.0     0.0   
...               ...   ...     ...   ...     ...   ...   ...   ...     ...   
1897335  200500087953     7       0   3.0     B01   1.0   1.0   1.0     0.0   
1897336  200500087953     2       0   1.0     A01   1.0   4.0   1.0     5.0   
1897337  200500087953     7       0   1.0     C01   1.0   1.0   1.0     0.0   
1897338  200500087954     7       0   0.0     A01   1.0   4.0   1.0     0.0   
1897339  200500087954     7       0   0.0     A01   2.0   4.0   2.0     0.0   

         secu  ...   com    dep  catr  circ  nbv  p

In [12]:
missing_values_table(accidents)

Your selected dataframe has 30 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [13]:
accidents.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1897340 entries, 0 to 1897339
Data columns (total 30 columns):
 #   Column   Dtype  
---  ------   -----  
 0   Num_Acc  int64  
 1   catv     int64  
 2   occutc   int64  
 3   choc     float64
 4   num_veh  object 
 5   catu     float64
 6   grav     float64
 7   sexe     float64
 8   trajet   float64
 9   secu     float64
 10  an_nais  float64
 11  an       float64
 12  mois     float64
 13  jour     float64
 14  hrmn     float64
 15  lum      float64
 16  agg      float64
 17  int      float64
 18  atm      float64
 19  col      float64
 20  com      float64
 21  dep      float64
 22  catr     float64
 23  circ     float64
 24  nbv      float64
 25  prof     float64
 26  plan     float64
 27  surf     float64
 28  infra    float64
 29  situ     float64
dtypes: float64(26), int64(3), object(1)
memory usage: 448.7+ MB


## One Hot Encoding

In [17]:
df_ohe = accidents.drop(['Num_Acc','jour','hrmn','com','num_veh','dep','an_nais','catv','occutc','agg'], axis = 1)
df_ohe = df_ohe.astype(float)
df_ohe

,choc,catu,grav,sexe,trajet,secu,an,mois,lum,int,atm,col,catr,circ,nbv,prof,plan,surf,infra,situ
0,1.0,1.0,1.0,2.0,0.0,11.0,16.0,2.0,1.0,1.0,8.0,3.0,3.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0
1,7.0,1.0,3.0,1.0,9.0,21.0,16.0,2.0,1.0,1.0,8.0,3.0,3.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0
2,1.0,1.0,3.0,1.0,5.0,11.0,16.0,3.0,1.0,6.0,1.0,6.0,3.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0
3,1.0,2.0,3.0,1.0,0.0,11.0,16.0,3.0,1.0,6.0,1.0,6.0,3.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0
4,1.0,2.0,3.0,2.0,0.0,11.0,16.0,3.0,1.0,6.0,1.0,6.0,3.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1897335,3.0,1.0,1.0,1.0,0.0,13.0,5.0,10.0,1.0,1.0,1.0,3.0,4.0,2.0,2.0,1.0,1.0,1.0,0.0,1.0
1897336,1.0,1.0,4.0,1.0,5.0,23.0,5.0,10.0,1.0,1.0,1.0,3.0,4.0,2.0,2.0,1.0,1.0,1.0,0.0,1.0
1897337,1.0,1.0,1.0,1.0,0.0,13.0,5.0,10.0,1.0,1.0,1.0,3.0,4.0,2.0,2.0,1.0,1.0,1.0,0.0,1.0
1897338,0.0,1.0,4.0,1.0,0.0,13.0,5.0,10.0,1.0,1.0,1.0,3.0,4.0,2.0,2.0,1.0,1.0,1.0,0.0,1.0


In [16]:
df_ohe.shape

(1897340, 21)

In [19]:
L = ['an', 'mois', 'lum', 'int', 'atm', 'col', 'catr', 'circ', 'nbv', 'prof',
       'plan', 'surf', 'infra', 'situ', 'catu', 'grav', 'sexe', 'trajet',
       'secu', 'choc']


for i in L:
    a = pd.get_dummies(df_ohe[i])
    print(a)

         5.0   6.0   7.0   8.0   9.0   10.0  11.0  12.0  13.0  14.0  15.0  \
0           0     0     0     0     0     0     0     0     0     0     0   
1           0     0     0     0     0     0     0     0     0     0     0   
2           0     0     0     0     0     0     0     0     0     0     0   
3           0     0     0     0     0     0     0     0     0     0     0   
4           0     0     0     0     0     0     0     0     0     0     0   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
1897335     1     0     0     0     0     0     0     0     0     0     0   
1897336     1     0     0     0     0     0     0     0     0     0     0   
1897337     1     0     0     0     0     0     0     0     0     0     0   
1897338     1     0     0     0     0     0     0     0     0     0     0   
1897339     1     0     0     0     0     0     0     0     0     0     0   

         16.0  
0           1  
1           1  
2           1  
3          

         0.0  1.0  2.0  3.0  4.0  5.0
0          0    1    0    0    0    0
1          0    1    0    0    0    0
2          0    1    0    0    0    0
3          0    1    0    0    0    0
4          0    1    0    0    0    0
...      ...  ...  ...  ...  ...  ...
1897335    0    1    0    0    0    0
1897336    0    1    0    0    0    0
1897337    0    1    0    0    0    0
1897338    0    1    0    0    0    0
1897339    0    1    0    0    0    0

[1897340 rows x 6 columns]
         1.0  2.0  3.0  4.0
0          1    0    0    0
1          1    0    0    0
2          1    0    0    0
3          0    1    0    0
4          0    1    0    0
...      ...  ...  ...  ...
1897335    1    0    0    0
1897336    1    0    0    0
1897337    1    0    0    0
1897338    1    0    0    0
1897339    0    1    0    0

[1897340 rows x 4 columns]
         1.0  2.0  3.0  4.0
0          1    0    0    0
1          0    0    1    0
2          0    0    1    0
3          0    0    1    0
4          0